In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import time
import os
import pickle

from my_lib.utility import *

## Actividad 3: Algoritmo genético para resolver el problema del viajero frecuente

El problema es de tipo TSP con 18 ciudades del conteninte Americano. Por lo tanto existen $18!\$ soluciones posibles ($6402373705728000$), dada el basto espacio de busqueda la población de individuos se debe limitar a minimo 100 y máximo 500.

> Criterios para la evaluación
> - Implementar 2 métodos de cruza, PMX y algún otro método como OX o CX
> - Implementar 2 métodos de mutación, Scramble y Heuristic
> - Implementar un método de selección, cualquiera de los ya vistos
> - Implementar un método hibrido de paro
> - Implementar un méotodo elitista
> - Realizar las comparaciones y la discusión de los resultados

# Codificación del dominio

In [2]:
nombre_ciudades = np.array(["Mexcio", "Miami", "Quito", "San Salvador", "Mendoza","Guadalajara"])

id_ciudades = np.arange(nombre_ciudades.shape[0])
bin_ciudades = binarizar(datos=id_ciudades,nbites=3)

ciudades = np.zeros((nombre_ciudades.shape[0],4),dtype="object")
ciudades[:,0] = nombre_ciudades
ciudades[:,1] = id_ciudades
ciudades[:,2] = bin_ciudades
ciudades[:,3] = np.zeros((nombre_ciudades.shape[0]))
print(ciudades)

[['Mexcio' 0 '000' 0.0]
 ['Miami' 1 '001' 0.0]
 ['Quito' 2 '010' 0.0]
 ['San Salvador' 3 '011' 0.0]
 ['Mendoza' 4 '100' 0.0]
 ['Guadalajara' 5 '101' 0.0]]


## Matriz de distancias

In [3]:
# Matriz de disntancias

distancias = np.array(
    [
        [0, 2270.05, 3714.66, 1778.27, 7223.16, 337.03],
        [2270.05, 0, 2882.57, 1627.62, 6578.78, 2425.29],
        [3714.66, 2882.57, 0, 1938.05, 3780.05, 3551.96],
        [1778.27, 1627.62, 1938.05, 0, 5567.96, 1681.97],
        [7223.16, 6578.78, 3780.05, 5567.96, 0, 6971.44],
        [337.03, 2425.29, 3551.96, 1681.97, 6971.44, 0],
    ]
)

print(distancias)

[[   0.   2270.05 3714.66 1778.27 7223.16  337.03]
 [2270.05    0.   2882.57 1627.62 6578.78 2425.29]
 [3714.66 2882.57    0.   1938.05 3780.05 3551.96]
 [1778.27 1627.62 1938.05    0.   5567.96 1681.97]
 [7223.16 6578.78 3780.05 5567.96    0.   6971.44]
 [ 337.03 2425.29 3551.96 1681.97 6971.44    0.  ]]


# Generación de población

Para poder realizar una buena comparación con las distintas combinaciones de operadores genéticos se trabajará con 5 poblaciones iniciales, las cuales son las mismas parar cada experimento


Si ya existe el archivo de poblaciones no hay necesidad de crear nuevamente las poblaciones iniciales

In [4]:
POB_FILE = "files\poblaciones_2.pkl"

In [5]:
def get_poblaciones_iniciales(n=5, tam_pob=60):
    """
    Genera y carga las poblaciones iniciales para un algoritmo genético.

    Si el archivo de poblaciones no existe, la función lo crea generando las poblaciones iniciales y guardándolas en el archivo.
    Si el archivo ya existe, la función lo carga y devuelve las poblaciones.

    Parámetros:
        n (int): Número de poblaciones iniciales a generar (por defecto, 5).
        tam_pob (int): Tamaño de cada población (por defecto, 60).

    Valor de retorno:
        list: Lista de poblaciones iniciales, donde cada población es una matriz de numpy con 4 columnas: ruta, aptitud, individuo y aptitud.
    """
    poblaciones = []
    POB_PATH = os.path.join("..", POB_FILE)
    if not os.path.exists(POB_PATH):
        print("El archivo de poblaciones no existse, creando archivo")
        
        for i in range(n):
            recorrido = generar_poblacion_perm(ciudades, tam_pob)
            
            poblacion = np.zeros((recorrido.shape[0], 4), dtype="object")
            poblacion[:, 0] = [c.astype(int) for c in recorrido]  # Ruta
            poblacion[:, 1] = np.ones((recorrido.shape[0]))
            poblacion[:, 2] = [individuo_toString(c.astype(int)) for c in recorrido]
            poblacion[:, 3] = np.zeros((recorrido.shape[0]))  # Aptitud
        
            poblaciones.append(poblacion)
        
        try:
            with open(POB_PATH, "wb") as f:
                pickle.dump(poblaciones,f)
            print("Archivo de poblaciones creado correctamente")
        except Exception as e:
            print(f"Error al guardar archivo: {e}")
            
    else:
        try:
            with open(POB_PATH, "rb") as f:
                poblaciones = pickle.load(f)
            print("Archivo de poblaciones cargado correctamente")
        except Exception as e:
            print(f"Error al cargar archivo: {e}")
    
    return poblaciones

In [6]:
pobs = get_poblaciones_iniciales(n=5,tam_pob=80)
print(len(pobs))

Archivo de poblaciones cargado correctamente
5


# Función objetivo

In [7]:
def y(ciudad1,ciudad2):
    return distancias[ciudad1,ciudad2]

def evaluar(poblacion):
    evaluacion = np.zeros((poblacion.shape[0]))
    for idx,fila in enumerate(poblacion):
        individuo = fila[0]
        n = len(individuo) - 1
        aptitud = 0
        
        for i in range (n,-1,-1):
            c1 = int(individuo[i]) #Id de la ciudad
            c2 = int(individuo[i-1])
            distancia = y(c1,c2)
            aptitud += distancia
           # print(f"Distancia entre {ciudades[c1]} y {ciudades[c2]}: {distancia}")
        evaluacion[idx] = np.around(aptitud,4)
        
    return evaluacion



# Experimento combinacion PMX, y Heuristic mutation

En este experimento a partir de la población incial, tomar en cuenta que hay 5 poblaciones inciales con las cuales debe realizarse el experimento, se obtienen nuevas generaciones de individuos y una vez alcanza el 80% de individuos aptos, esto quiere decir que los individuso deben superar más del 0.8 de aptitud, o se llega un 10 generaciones el algoritmo se detiene.

**Condiciones de termino:**

* Epsilon:
    * Cuando el 80% de la población alcanza una aptitud mayor de 0.8
* Generaciones:
    * Cuando se llega a la generación número 10

**Operadores:**
* Método de competencia genética para la creación de las poblaciones
* Operador de selección *Ruleta*
* Operador de cruce *PMX*
* Operador de mutación  *Hueristico*

## Evaluar y Ordenar la poblacion

La función de ordenar población ordena de mayor a menor.

Este problema es un problema para encontrar las rutas con menor costo

In [8]:
MAX_GEN = 10

In [11]:
poblacion

array([[array([5, 0, 3, 4, 2, 1]), 1.0, '503421', 16771.17],
       [array([4, 2, 3, 0, 5, 1]), 1.0, '423051', 16837.47],
       [array([0, 3, 2, 4, 1, 5]), 1.0, '032415', 16837.47],
       [array([4, 2, 3, 1, 0, 5]), 1.0, '423105', 16924.24],
       [array([1, 3, 2, 4, 5, 0]), 1.0, '132450', 16924.24],
       [array([5, 0, 1, 3, 4, 2]), 1.0, '501342', 17134.67],
       [array([4, 2, 5, 0, 1, 3]), 1.0, '425013', 17134.67],
       [array([3, 1, 5, 0, 4, 2]), 1.0, '315042', 17331.2],
       [array([3, 1, 5, 0, 4, 2]), 1.0, '315042', 17331.2],
       [array([4, 2, 3, 1, 5, 0]), 1.0, '423150', 17331.2],
       [array([2, 4, 0, 5, 1, 3]), 1.0, '240513', 17331.2],
       [array([4, 2, 1, 3, 0, 5]), 1.0, '421305', 17376.98],
       [array([4, 1, 3, 0, 5, 2]), 1.0, '413052', 17653.71],
       [array([4, 2, 5, 0, 3, 1]), 1.0, '425031', 17653.71],
       [array([4, 2, 5, 0, 3, 1]), 1.0, '425031', 17653.71],
       [array([4, 1, 3, 0, 5, 2]), 1.0, '413052', 17653.71],
       [array([2, 3, 0, 1, 5

In [9]:
#Tiempo del experimento total
start_exp_time = time.time()
for i in range(len(pobs)): 
    #Cargar población incial
    print(f"Población inicial: {i}")
    poblacion = pobs[i]
    #evaluar la población
    pob_aptitud = evaluar(poblacion)
    poblacion[:,3] = pob_aptitud
    #La población esta ordenada de mayor a menor, por lo tanto los últimos indiviudos siempre son los más aptos 
    poblacion = ordenar_poblacion(poblacion,reverse=True)
    

    # print(poblacion.shape)
    # print(poblacion[-10:])
    
    
    #Experimentación
    GEN = 0
    GENERACIONES = []
    HISTDF = []
     
    GENERACIONES.append(poblacion)
    
    start_time = time.time()
    while True:
        #poblacion actual
        print(f"Generación: {GEN}")
        print(np.mean(poblacion[:,3]))
        
        if GEN >= MAX_GEN or paro_epsilon(poblacion,14000,0.9,opt=1):
            print("Convergencia alcanzada")
            break
        
        else:
            
            #Obtener siguiente generacion
            #Selección y cruza
            hijos = get_next_gen(poblacion,op_seleccion=0,op_cruza=2,mode="min")
            print(hijos)
            
            GEN += 1
            
            
            
    end_time = time.time()
    elapsed_time = start_time - end_time
    print(f"Tiempo de ejecucion iteracion {i}: {elapsed_time} segundos")
    break


end_exp_time = time.time()
elapsed_exp_time = end_exp_time - start_exp_time

Población inicial: 0
Generación: 0
20907.188374999987
[[array([5, 3, 1, 0, 4, 2]), array(['501342', '230154'], dtype=object)], [array([2, 0, 3, 1, 5, 4]), array(['501342', '230154'], dtype=object)], [array([4, 2, 3, 0, 1, 5]), array(['032415', '423051'], dtype=object)], [array([0, 3, 2, 4, 5, 1]), array(['032415', '423051'], dtype=object)], [array([5, 2, 3, 4, 0, 1]), array(['503421', '524013'], dtype=object)], [array([5, 0, 4, 2, 1, 3]), array(['503421', '524013'], dtype=object)], [array([3, 0, 2, 1, 4, 5]), array(['320145', '102453'], dtype=object)], [array([1, 2, 0, 4, 5, 3]), array(['320145', '102453'], dtype=object)], [array([1, 4, 3, 0, 5, 2]), array(['025134', '143052'], dtype=object)], [array([0, 2, 5, 1, 3, 4]), array(['025134', '143052'], dtype=object)], [array([0, 1, 4, 5, 3, 2]), array(['045132', '314052'], dtype=object)], [array([3, 4, 5, 0, 1, 2]), array(['045132', '314052'], dtype=object)], [array([3, 1, 4, 2, 5, 0]), array(['314052', '534201'], dtype=object)], [array([5

In [10]:
print(f"Tiempo de experimentación: {elapsed_exp_time}")

Tiempo de experimentación: 0.4429662227630615
